<a href="https://colab.research.google.com/github/navrgithub/NLP_Authorship_Attribution/blob/main/final_bilstm_task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/NLP group project/data')

In [ ]:
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.2 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',tokenizer_language='en_core_web_sm', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('final_task3_data.csv')

In [ ]:
df.head()

,Generation,label
0,latest headlines on cnn business tl;dr the u.s...,ctrl
1,china wants to take a victory lap over its han...,ctrl
2,coronavirus disinformation creates challenges ...,ctrl
3,china coronavirus: eating wild animals made il...,ctrl
4,china's economy could shrink for the first tim...,ctrl


In [ ]:
df[['text']] = df[['Generation']]. astype(str)

In [ ]:
df = df[['text','label']]

In [ ]:
TEXT = data.Field(tokenize = 'spacy', tokenizer_language='en_core_web_sm',  include_lengths = True)
LABEL = data.LabelField(sequential=False, use_vocab=False, dtype = torch.float)

pt = [TEXT.preprocess(x) for x in df['text']]

TEXT.build_vocab(
    preprocessed_text, 
    max_size = 30000,
    vectors='glove.6B.50d',
    unk_init = torch.Tensor.normal_
)

In [ ]:
label_to_index= {label: index for index, label in enumerate(df['label'].unique())}

In [ ]:
df['label_new'] = df['label'].apply(lambda y: label_to_index[y])

class DataFrameDataset(data.Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                data.Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], fields)

In [ ]:
df_new=df[['text','label_new']]

In [ ]:
examples=[]
for i,row in df_new.iterrows():
  examples.append(data.Example.fromlist(row, fields=[('text', TEXT), ('label', LABEL)]))

In [ ]:
itrr=0
for i in examples:
  if type(i.label) != int:
    itrr+=1

In [ ]:
dataset = data.Dataset(examples, fields=[('text', TEXT), ('label', LABEL)])

In [ ]:
train_dataset, test_dataset, valid_dataset = dataset.split(split_ratio=[0.65, 0.15, 0.2], stratified=True, strata_field='label')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("We are working with ", device)
# device = 'cpu'
train_iter, test_iter, valid_iter = data.BucketIterator.splits(
    datasets=(train_dataset, test_dataset, valid_dataset), 
    batch_size=32,
    sort_within_batch = True,
    sort_key=lambda x: len(x.text),
    device=device
)

We are working with  cuda


In [ ]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        super().__init__()
        #1. Initialize Embedding Layer
        self.embedding = nn.Embedding(num_embeddings = vocab_size, embedding_dim = embedding_dim, padding_idx = pad_idx)
        #2. Initialize LSTM layer
        self.lstm = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_dim, num_layers = n_layers, dropout = dropout, bidirectional = bidirectional)
        #3. Initialize a fully connected layer with Linear transformation
        self.fc = nn.Linear(hidden_dim * n_layers, output_dim)
        #4. Initialize Dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        #1. Apply embedding layer that matches each word to its vector and apply dropout. Dim [sent_len, batch_size, emb_dim]
        embedding = self.embedding(text)
        embedding_dropout = self.dropout(embedding)
        #2. Run the LSTM along the sentences of length sent_len. #output = [sent len, batch size, hid dim * num directions]; #hidden = [num layers * num directions, batch size, hid dim]
        output, (hidden,cell) = self.lstm(embedding_dropout)
        #3. Concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers and apply dropout
        concat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        hidden = self.dropout(concat)     
        return self.fc(hidden)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 126
OUTPUT_DIM = 11
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTM(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT,
            PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,065,347 trainable parameters


In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([30002, 50])


In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  0.7372,  0.3910,  0.5158],
        [ 0.1825, -0.0737,  0.3147,  ...,  1.3226, -2.6796, -0.1297],
        [ 0.1516,  0.3018, -0.1676,  ..., -0.3565,  0.0164,  0.1022],
        ...,
        [-2.4447,  0.3625,  0.0473,  ..., -0.7721, -1.5739,  0.8893],
        [-1.0916, -0.6359,  0.8588,  ...,  0.9888,  0.3444, -0.6899],
        [ 1.2178, -1.4262,  0.5219,  ..., -1.3257,  0.9352, -2.3603]])

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1516,  0.3018, -0.1676,  ..., -0.3565,  0.0164,  0.1022],
        ...,
        [-2.4447,  0.3625,  0.0473,  ..., -0.7721, -1.5739,  0.8893],
        [-1.0916, -0.6359,  0.8588,  ...,  0.9888,  0.3444, -0.6899],
        [ 1.2178, -1.4262,  0.5219,  ..., -1.3257,  0.9352, -2.3603]])


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.CrossEntropyLoss()

# device='cpu'
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def multiclass_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    # get the index of the highest prediction for each instance
    _, preds = torch.max(preds, dim=1)
    correct = (preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc


In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        # print(batch.text)
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)

        target = batch.label
        target = target.type(torch.LongTensor)

        # print(target)

        # print(predictions)
        
        loss = criterion(predictions.to(device), target.to(device))
        
        acc = multiclass_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            
            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            target = batch.label
            target = target.type(torch.LongTensor)
            
            loss = criterion(predictions.to(device), target.to(device))
            
            acc = multiclass_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()



    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 50
path = F"bilstm_model_task3.pt"
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 8s
	Train Loss: 1.845 | Train Acc: 30.50%
	 Val. Loss: 1.528 |  Val. Acc: 42.12%
Epoch: 02 | Epoch Time: 0m 7s
	Train Loss: 1.365 | Train Acc: 47.37%
	 Val. Loss: 1.244 |  Val. Acc: 54.00%
Epoch: 03 | Epoch Time: 0m 6s
	Train Loss: 1.135 | Train Acc: 55.51%
	 Val. Loss: 0.876 |  Val. Acc: 64.50%
Epoch: 04 | Epoch Time: 0m 7s
	Train Loss: 0.922 | Train Acc: 63.64%
	 Val. Loss: 0.872 |  Val. Acc: 66.50%
Epoch: 05 | Epoch Time: 0m 7s
	Train Loss: 0.730 | Train Acc: 70.45%
	 Val. Loss: 1.155 |  Val. Acc: 62.25%
Epoch: 06 | Epoch Time: 0m 7s
	Train Loss: 0.718 | Train Acc: 71.30%
	 Val. Loss: 0.574 |  Val. Acc: 73.19%
Epoch: 07 | Epoch Time: 0m 7s
	Train Loss: 0.541 | Train Acc: 76.97%
	 Val. Loss: 0.559 |  Val. Acc: 77.81%
Epoch: 08 | Epoch Time: 0m 7s
	Train Loss: 0.432 | Train Acc: 81.49%
	 Val. Loss: 0.493 |  Val. Acc: 80.50%
Epoch: 09 | Epoch Time: 0m 7s
	Train Loss: 0.458 | Train Acc: 80.95%
	 Val. Loss: 0.412 |  Val. Acc: 84.00%
Epoch: 10 | Epoch Time: 0m 7

In [ ]:
test_loss, test_acc = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.386 | Test Acc: 89.68%


In [ ]:
import pandas as pd


In [ ]:
df_reddit = pd.read_csv('/content/gdrive/MyDrive/NLP/reddit_data_gpt3.csv')

In [ ]:
df_reddit.head()

,Unnamed: 0,Title,Body,Generated Text
0,0,"CMV: Demanding ""Asian people"" to have better g...",Edit:I talk about the fact that Asian applican...,"This is an inherently racist statement, becaus..."
1,1,CMV: Eating your lunch on the toilet is a disg...,I have a coworker who does this and it's prett...,This is a disgusting habit. Eating your lunch ...
2,2,CMV: Megamind was morally justified in catfish...,Hey guys! Megamind is one of my favorite movie...,Megamind was justified in catfishing Roxanne R...
3,3,CMV: I don't see why a chef or cook needs to u...,I think that this is something I once heard Go...,It's not necessary.I totally agree. I think t...
4,4,CMV: The government should not be the arbiter ...,One thing that I have found lacking in the gay...,"This is a difficult question. On one hand, the..."


In [ ]:
df_reddit['text'] = df_reddit['Title']+df_reddit['Body']+df_reddit['Generated Text']

In [ ]:
df_reddit = df_reddit [['text']]
df_reddit['label']=9

<ipython-input-80-7894b9cf1a32>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reddit['label']=9


In [ ]:
ltoi

{'grover': 0,
 'ctrl': 1,
 'instructgpt': 2,
 'pplm': 3,
 'gpt2': 4,
 'xlnet': 5,
 'gpt': 6,
 'xlm': 7,
 'human': 8,
 'gpt3': 9,
 'fair': 10}

In [ ]:
df_reddit.head()

,text,label
0,"CMV: Demanding ""Asian people"" to have better g...",9
1,CMV: Eating your lunch on the toilet is a disg...,9
2,CMV: Megamind was morally justified in catfish...,9
3,CMV: I don't see why a chef or cook needs to u...,9
4,CMV: The government should not be the arbiter ...,9


In [ ]:
examples=[]
for i,row in df_reddit.iterrows():
  # print(example)
  examples.append(data.Example.fromlist(row, fields=[('text', TEXT), ('label', LABEL)]))

In [ ]:
# examples

In [ ]:
examples=[]
for i,row in df_reddit.iterrows():
  # print(example)
  examples.append(data.Example.fromlist(row, fields=[('text', TEXT), ('label', LABEL)]))

dataset = data.Dataset(examples, fields=[('text', TEXT), ('label', LABEL)])
train_dataset, test_dataset, valid_dataset = dataset.split(split_ratio=[0.65, 0.15, 0.2], stratified=True, strata_field='label')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device = 'cpu'
print("We are working with ", device)
train_iter, test_iter, valid_iter = data.BucketIterator.splits(
    datasets=(train_dataset, test_dataset, valid_dataset), 
    batch_size=32,
    sort_within_batch = True,
    sort_key=lambda x: len(x.text),
    device=device
)

We are working with  cpu


In [ ]:
def evaluate_reddit(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:   
            text, text_lengths = batch.text
            predictions = model(text, text_lengths).squeeze(1)
            target = batch.label
            target = target.type(torch.LongTensor)
            
            loss = criterion(predictions.to(device), target.to(device))
            
            acc = multiclass_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()



    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
model.load_state_dict(torch.load(path))


<All keys matched successfully>

In [ ]:
loss,acc = evaluate_reddit(model,train_iter,criterion)

In [ ]:
acc*100

9.114583333333332

In [ ]:
loss,acc = evaluate_reddit(model,test_iter,criterion)

In [ ]:
acc*100

7.8125